# Create Json mask file to choose the appropriate threshold

Labelled exported as geoJSON:

* for QuPath coordinates need to be integers (pixels)
* coordinates need to be (x,y) and in form of a list (not nd-array)
* export of cells (around 10k cells) will take a while. Import to QuPath fast, as long as * no hierarchy is resolved

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os, sys

other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(other_dir_path)

print(other_dir_path)
import IMCprocess.spatial_analysis.spatial_analysis as sa
import IMCprocess.spatial_analysis.combi_enrich as ce
import IMCprocess.utils.utils as ut

# I have to use custom Img_anndata code
import Img_anndata as utan

/Users/pattama/Desktop/UiB/cancer/git_repo/IMCprocess


## Import mask and level1 annotated object

In [2]:
input_mask = 'sample_data/Patient1_001.tiff'
output_dir = 'output'
level1_file = 'sample_data/00hanna_adata_pilottma1_obj_Level1.pkl'

In [3]:
Img_obj_level1 = ut.read_pickle_obj(level1_file)

In [4]:
Img_obj_level1.adata_dict

{'level1': AnnData object with n_obs × n_vars = 14300 × 36
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities'}

## Create epithelial anndata

In [5]:
epi_adata = Img_obj_level1.adata_dict['level1'][
    Img_obj_level1.adata_dict[
        'level1'].obs['level1_annotated'] == 'Ep']

In [7]:
# Check marker column name
epi_adata.var_names

Index(['Bi209_pSTAT3_', 'Dy161_pSMAD2_', 'Dy162_CD8_', 'Dy163_ER_',
       'Dy164_CD20_', 'Er166_HER2_', 'Er167_GATA3_', 'Er168_Ki67_',
       'Er170_CD3_', 'Eu151_CD3134_', 'Eu153_CD44_', 'Gd155_FoxP3_',
       'Gd156_CD4_', 'Gd158_Ecad_', 'Gd160_PDGFRa_', 'In113_ADHL1_',
       'In115_GammaCat_', 'La139_MCAM_', 'Nd142_CTLA4_', 'Nd143_Vim_',
       'Nd144_YAP1_', 'Nd145_CD45_', 'Nd146_CD16_', 'Nd148_PanCk_',
       'Nd150_PDL1_', 'Pr141_aSMA_', 'Sm147_CD163_', 'Sm149_CK5_',
       'Sm152_FAP_', 'Sm154_CD11c_', 'Tb159_CD68_', 'Tm169_CD24_',
       'Yb171_PDGFRb_', 'Yb172_ClCasp3_', 'Yb173_GranzymeB_', 'Yb174_CK818_'],
      dtype='object')

## label 1, 0 for positive and negative cells from threshold

In [8]:
Ki67_df = epi_adata.to_df()[['Er168_Ki67_']]
Vim_df = epi_adata.to_df()[['Nd143_Vim_']]

### range threshold from 0.2 - 0.5

In [10]:
def binary_df(df, threshold):
    df_binary = df.applymap(lambda x: 1 if x >= threshold else 0)
    df_binary.columns = [col + '_'+ str(round(threshold,2)) for col in df.columns]
    return(df_binary)

In [11]:
Ki67_bi_ls = [binary_df(Ki67_df, i) for i in np.arange(0.2, 0.51, 0.05)]
Vim_bi_ls = [binary_df(Vim_df, i) for i in np.arange(0.2, 0.51, 0.05)]

In [16]:
Vim_bi_concat = pd.concat(Vim_bi_ls, axis=1).reset_index()
Ki67_bi_concat = pd.concat(Ki67_bi_ls, axis=1).reset_index()

In [17]:
Vim_bi_concat.head()

,Cell_id,Nd143_Vim__0.2,Nd143_Vim__0.25,Nd143_Vim__0.3,Nd143_Vim__0.35,Nd143_Vim__0.4,Nd143_Vim__0.45,Nd143_Vim__0.5
0,14777_0,0,0,0,0,0,0,0
1,14778_0,1,1,1,1,0,0,0
2,14779_0,0,0,0,0,0,0,0
3,14780_0,1,1,1,1,0,0,0
4,14782_0,1,1,1,1,0,0,0


In [24]:
def cell_neg_pos_dict(df, col):
    cell_neg_ind = np.array(df[df[col] == 0].index) + 1
    cell_pos_ind = np.array(df[df[col] == 1].index) + 1
    label_dict = {('neg_'+col): cell_neg_ind,
                  ('pos_'+col): cell_pos_ind}
    return(label_dict)

In [ ]:
label_dict = {('neg_'+col): cell_neg_ind,
              ('pos_'+col): cell_pos_ind}
        